In [1]:
spark

In [2]:
import os, json
from pprint import pprint
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import pandas as pd

import datetime
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col,lit
from pyspark.sql.window import Window

from functools import reduce

import re


In [3]:
os.chdir('../')
os.getcwd()

'/home/gaborotta/workspace/predict_roto6'

In [4]:
# cred = credentials.Certificate('./serviceAccount.json')
# firebase_admin.initialize_app(cred)

# db = firestore.client()

In [5]:

# users_ref = db.collection(u'number_results')
# docs = users_ref.stream()

# doc_list = [doc.to_dict() for doc in docs]



In [6]:
# pprint(doc_list[0])

In [7]:
# with open('data/number_results.json', 'w') as f:
#     json.dump(doc_list, f, indent=4)

# よみこみ

In [8]:
with open('data/number_results.json') as f:
    number_results = pd.read_json(f)
number_results.head(2)

,fifth_hits,third_hits,second_hits,first_hits,fifth_amount,date,first_amount,carryover,number_B,second_amount,fourth_hits,numbers,title,third_amount,fourth_amount
0,211620,303,8,0,1000,2015-09-07,0,268345233,37,16056900,13791,"[1, 15, 21, 22, 26, 29]",ロト6第1000回,508700,9800
1,231019,156,5,1,1000,2015-09-10,400000000,7153015,2,24983100,14036,"[9, 12, 15, 16, 18, 35]",ロト6第1001回,585500,9400


In [9]:
df= spark.createDataFrame(number_results)

In [15]:
df.show(2)

+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+--------------------+-------------+------------+-------------+
|fifth_hits|third_hits|second_hits|first_hits|fifth_amount|      date|first_amount|carryover|number_B|second_amount|fourth_hits|             numbers|        title|third_amount|fourth_amount|
+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+--------------------+-------------+------------+-------------+
|    211620|       303|          8|         0|        1000|2015-09-07|           0|268345233|      37|     16056900|      13791|[1, 15, 21, 22, 2...|ロト6第1000回|      508700|         9800|
|    231019|       156|          5|         1|        1000|2015-09-10|   400000000|  7153015|       2|     24983100|      14036|[9, 12, 15, 16, 1...|ロト6第1001回|      585500|         9400|
+----------+----------+-----------+----------+-------

In [16]:
os.getcwd()

'/home/gaborotta/workspace/predict_roto6'

In [6]:
out_dir = os.getcwd() + 'data/'

In [35]:
path = out_dir + 'number_results'
df.write.mode('ignore').parquet(path)
number_results = spark.read.parquet(path)

In [36]:
number_results.show(2)
number_results.count()

+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+--------------------+------------+------------+-------------+
|fifth_hits|third_hits|second_hits|first_hits|fifth_amount|      date|first_amount|carryover|number_B|second_amount|fourth_hits|             numbers|       title|third_amount|fourth_amount|
+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+--------------------+------------+------------+-------------+
|    511511|       719|         28|         3|        1000|2004-03-25|    95437500|        0|      23|      9202800|      31445|[3, 6, 11, 13, 15...|ロト6第179回|      430000|         8600|
|    264631|       209|          7|         0|        1000|2001-01-25|           0|249208924|      11|     32040800|      14265|[14, 20, 25, 34, ...| ロト6第17回|     1287700|        16600|
+----------+----------+-----------+----------+------------

1563

# 展開

In [49]:
number_results_ed = (
    number_results
    .withColumn('numbers',F.explode('numbers'))
    .withColumn('date',F.to_date('date'))
    .withColumn('title_num',F.element_at( F.split(col('title'),'[第回]'),2).cast('int'))
)

In [7]:
path = out_dir + 'number_results_ed'
number_results_ed.write.mode('overwrite').parquet(path)
number_results_ed = spark.read.parquet(path)

NameError: name 'number_results_ed' is not defined

In [51]:
number_results_ed.show(1)

+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+-------+------------+------------+-------------+---------+
|fifth_hits|third_hits|second_hits|first_hits|fifth_amount|      date|first_amount|carryover|number_B|second_amount|fourth_hits|numbers|       title|third_amount|fourth_amount|title_num|
+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+-------+------------+------------+-------------+---------+
|    232486|       364|         12|         0|        1000|2014-07-14|           0|116551216|      35|      8741200|      15752|      4|ロト6第881回|      345800|         7000|      881|
+----------+----------+-----------+----------+------------+----------+------------+---------+--------+-------------+-----------+-------+------------+------------+-------------+---------+
only showing top 1 row



In [52]:
number_results_ed.printSchema()

root
 |-- fifth_hits: long (nullable = true)
 |-- third_hits: long (nullable = true)
 |-- second_hits: long (nullable = true)
 |-- first_hits: long (nullable = true)
 |-- fifth_amount: long (nullable = true)
 |-- date: date (nullable = true)
 |-- first_amount: long (nullable = true)
 |-- carryover: long (nullable = true)
 |-- number_B: long (nullable = true)
 |-- second_amount: long (nullable = true)
 |-- fourth_hits: long (nullable = true)
 |-- numbers: long (nullable = true)
 |-- title: string (nullable = true)
 |-- third_amount: long (nullable = true)
 |-- fourth_amount: long (nullable = true)
 |-- title_num: integer (nullable = true)



# 数字ごとの頻出度を計算

In [55]:
res = (
    number_results_ed
    .select(
        F.countDistinct('title').alias('count'),
        F.max('title_num').alias('latest_num'),
    ).take(1)[0]
)
roto_count = res ['count']
latest_num = res ['latest_num']

In [56]:
res

Row(count=1563, latest_num=1563)

In [14]:
def calc_number_count(mart,num_str):
    return (
        mart
        .groupBy('numbers')
        .agg(F.count(lit(1)).alias(f'count_{num_str}'))
#         .withColumn(f'rate_{num_str}',col(f'count_{num_str}')/lit(roto_count))
    )
    

In [80]:
latest_num - 6

1557

In [105]:
number_results_ed = number_results_ed.where(col('title_num') <= latest_num - 1 )
latest_num = latest_num -1
number_counts_list = [
    calc_number_count(number_results_ed,'all'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 2) ),'3'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 6) ),'7'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 9) ),'10'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 19) ),'20'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 29) ),'30'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 49) ),'50'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 99) ),'100'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 199) ),'200'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 499) ),'500'),
    calc_number_count(number_results_ed.where(col('title_num') >= lit(latest_num - 999) ),'1000'),
]


number_counts = reduce(
    lambda a,b: a.join(b,'numbers','left_outer'),
    number_counts_list
).fillna(0)

In [106]:
path = out_dir + 'number_counts'
number_counts.write.mode('overwrite').parquet(path)
number_counts = spark.read.parquet(path)

In [107]:
number_counts.orderBy(F.desc('rate_all')).show(100)

+-------+---------+-------------------+-------+--------------------+-------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+---------+--------------------+---------+--------------------+---------+--------------------+----------+-------------------+
|numbers|count_all|           rate_all|count_3|              rate_3|count_7|              rate_7|count_10|             rate_10|count_20|             rate_20|count_30|             rate_30|count_50|             rate_50|count_100|            rate_100|count_200|            rate_200|count_500|            rate_500|count_1000|          rate_1000|
+-------+---------+-------------------+-------+--------------------+-------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+---------+--------------------+---------+--------------------+---------+--------------------+------

In [123]:
num_strs = [
    'all','3','7','10','20','30','50','100','200','500','1000'
]


number_counts_ed = (
    number_counts
#     .withColumn(f'qtile_{num_str}',F.ntile(4).over(Window.orderBy(F.desc(f'rate_{num_str}'))))
)

for ns in num_strs:
    number_counts_ed = (
        number_counts_ed
        .withColumn(f'qtile_{ns}',F.ntile(10).over(Window.orderBy(F.desc(f'rate_{ns}'))))
    )

In [124]:
path = out_dir + 'number_counts_ed'
number_counts_ed.write.mode('overwrite').parquet(path)
number_counts_ed = spark.read.parquet(path)

In [125]:
number_counts_ed.show(100)

+-------+---------+-------------------+-------+--------------------+-------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+--------+--------------------+---------+--------------------+---------+--------------------+---------+--------------------+----------+-------------------+---------+-------+-------+--------+--------+--------+--------+---------+---------+---------+----------+
|numbers|count_all|           rate_all|count_3|              rate_3|count_7|              rate_7|count_10|             rate_10|count_20|             rate_20|count_30|             rate_30|count_50|             rate_50|count_100|            rate_100|count_200|            rate_200|count_500|            rate_500|count_1000|          rate_1000|qtile_all|qtile_3|qtile_7|qtile_10|qtile_20|qtile_30|qtile_50|qtile_100|qtile_200|qtile_500|qtile_1000|
+-------+---------+-------------------+-------+--------------------+-------+--------------------+--------+----

In [126]:
number_counts_ed.select('numbers','count_all',*[f'qtile_{x}' for x in num_strs]).orderBy('count_all','numbers').show(100)

+-------+---------+---------+-------+-------+--------+--------+--------+--------+---------+---------+---------+----------+
|numbers|count_all|qtile_all|qtile_3|qtile_7|qtile_10|qtile_20|qtile_30|qtile_50|qtile_100|qtile_200|qtile_500|qtile_1000|
+-------+---------+---------+-------+-------+--------+--------+--------+--------+---------+---------+---------+----------+
|      9|      197|       10|     10|     10|      10|      10|       7|       6|        2|        6|        4|        10|
|      7|      198|       10|      4|      4|       3|       5|       8|       9|        8|        8|        9|         8|
|     29|      199|       10|     10|     10|      10|      10|      10|      10|        9|        8|       10|         8|
|     13|      201|       10|     10|     10|      10|      10|      10|       9|        8|        9|       10|         9|
|     41|      204|        9|     10|     10|       8|       8|       6|       3|        3|        1|        6|         9|
|     34|      2

# 結合して集計

In [8]:
path = out_dir + 'number_results_ed'
number_results_ed = spark.read.parquet(path)
number_results_ed

DataFrame[fifth_hits: bigint, third_hits: bigint, second_hits: bigint, first_hits: bigint, fifth_amount: bigint, date: date, first_amount: bigint, carryover: bigint, number_B: bigint, second_amount: bigint, fourth_hits: bigint, numbers: bigint, title: string, third_amount: bigint, fourth_amount: bigint, title_num: int]

In [128]:
number_results_ed_ed = (
    number_results_ed
    .join(
        number_counts_ed
        .select('numbers',*[f'qtile_{x}' for x in num_strs]),
        'numbers','left_outer'
    )
)

path = out_dir + 'number_results_ed_ed'
number_results_ed_ed.write.mode('overwrite').parquet(path)
number_results_ed_ed = spark.read.parquet(path)

In [129]:
number_results_ed_ed.select(
    'title',
    'title_num',
    'numbers',
    *[f'qtile_{x}' for x in num_strs]
).orderBy(F.desc('title_num')).show(100)


+-------------+---------+-------+---------+-------+-------+--------+--------+--------+--------+---------+---------+---------+----------+
|        title|title_num|numbers|qtile_all|qtile_3|qtile_7|qtile_10|qtile_20|qtile_30|qtile_50|qtile_100|qtile_200|qtile_500|qtile_1000|
+-------------+---------+-------+---------+-------+-------+--------+--------+--------+--------+---------+---------+---------+----------+
|ロト6第1563回|     1563|     20|        3|      6|      5|       6|       9|       8|       8|        9|        5|        3|         5|
|ロト6第1563回|     1563|      1|        8|      9|      7|       7|       7|       4|       5|        3|        3|        2|         6|
|ロト6第1563回|     1563|     35|        8|      3|      3|       4|       4|       3|       4|        3|        5|        8|         9|
|ロト6第1563回|     1563|     37|        1|      4|      8|       8|       8|       1|       1|        1|        1|        4|         2|
|ロト6第1563回|     1563|     12|        3|      2|      2|  

In [130]:


for x in num_strs:
    (
        number_results_ed_ed
        .where(col('title_num')==1563)
        .groupBy('title','title_num')
        .pivot('numbers')
        .agg(F.avg(f'qtile_{x}'))
        .withColumn('num_str',lit(x))
    ).orderBy(F.desc('title_num')).show(5)

+-------------+---------+---+---+---+---+---+---+-------+
|        title|title_num|  1| 12| 20| 26| 35| 37|num_str|
+-------------+---------+---+---+---+---+---+---+-------+
|ロト6第1563回|     1563|8.0|3.0|3.0|6.0|8.0|1.0|    all|
+-------------+---------+---+---+---+---+---+---+-------+

+-------------+---------+---+---+---+---+---+---+-------+
|        title|title_num|  1| 12| 20| 26| 35| 37|num_str|
+-------------+---------+---+---+---+---+---+---+-------+
|ロト6第1563回|     1563|9.0|2.0|6.0|1.0|3.0|4.0|      3|
+-------------+---------+---+---+---+---+---+---+-------+

+-------------+---------+---+---+---+---+---+---+-------+
|        title|title_num|  1| 12| 20| 26| 35| 37|num_str|
+-------------+---------+---+---+---+---+---+---+-------+
|ロト6第1563回|     1563|7.0|2.0|5.0|1.0|3.0|8.0|      7|
+-------------+---------+---+---+---+---+---+---+-------+

+-------------+---------+---+---+---+---+---+---+-------+
|        title|title_num|  1| 12| 20| 26| 35| 37|num_str|
+-------------+--------

過去30回を参考にするようにしてみる？

 # 過去30回基準で全ての回を計算

In [5]:
out_dir = os.getcwd() + 'data/'
path = out_dir + 'number_results_ed'
number_results_ed = spark.read.parquet(path)
number_results_ed

DataFrame[fifth_hits: bigint, third_hits: bigint, second_hits: bigint, first_hits: bigint, fifth_amount: bigint, date: date, first_amount: bigint, carryover: bigint, number_B: bigint, second_amount: bigint, fourth_hits: bigint, numbers: bigint, title: string, third_amount: bigint, fourth_amount: bigint, title_num: int]

In [6]:
def calc_number_count(mart,num_str):
    return (
        mart
        .groupBy('numbers')
        .agg(F.count(lit(1)).alias(f'count_{num_str}'))
#         .withColumn(f'rate_{num_str}',col(f'count_{num_str}')/lit(roto_count))
    )
    

In [6]:
def get_num_str_res(latest_num=1563, ns=30, ntile=10):
#     num_strs = ['all',f'{ns}']
    mart = number_results_ed.where(col('title_num') <= latest_num - 1 )
    
    number_counts_list = [
        calc_number_count(mart,'all'),
        calc_number_count(mart.where(col('title_num') >= lit(latest_num - ns) ),f'{ns}'),
    ]

    number_counts = reduce(
        lambda a,b: a.join(b,'numbers','left_outer'),
        number_counts_list
    ).fillna(0)
    
    number_counts_ed = (
        number_counts
        .withColumn('qtile_all',F.ntile(ntile).over(Window.orderBy(F.desc(f'count_all'))))
        .withColumn('qtile',F.ntile(ntile).over(Window.orderBy(F.desc(f'count_{ns}'))))
    )
    
    number_results_ed_ed = (
        number_results_ed
        .join(
            number_counts_ed
            .select('numbers','qtile_all','qtile'),
            'numbers','left_outer'
        )
    )
    
#     sum_mart_2 = (
#         number_results_ed_ed
#         .where(col('title_num')==latest_num)
#         .groupBy('title','title_num','qtile')
#         .agg(
#             F.count(lit(1)).alias('count'),
#         )
#     )
#     path = out_dir + f'sum_mart_2/latest_num={latest_num}/ns={ns}/ntile={ntile}'
# #     print(path)
#     sum_mart_2.write.mode('overwrite').parquet(path)
#     sum_mart_2 = spark.read.parquet(path)
    
    sum_mart = (
        number_results_ed_ed
        .where(col('title_num')==latest_num)
        .groupBy('title','title_num')
        .agg(
            *[
                F.count(F.when(col(f'qtile')==qx,1)).alias(f'count_{qx}')
                for qx in range(1,ntile+1)
            ]
        )
    )
    
#     sum_mart = (
#         number_results_ed_ed
#         .where(col('title_num')==latest_num)
#         .groupBy('title','title_num')
#         .pivot('numbers')
#         .agg(F.avg(f'qtile_{ns}'))
#         .withColumn('num_str',lit(ns))
#     ).orderBy(F.desc('title_num'))
    
    path = out_dir + f'sum_mart_{latest_num}_{ns}_{ntile}'
#     print(path)
    sum_mart.write.mode('overwrite').parquet(path)
    sum_mart = spark.read.parquet(path)
        
    return sum_mart

In [19]:
tl = [
    5,
    10,
    30,
    50,
    100,
    300,
    500
]

for t in tl:
    print(t)
    smart_list = [
        get_num_str_res(latest_num=x, ns=t, ntile=14)
        for x in range(1563,1463,-1)    
    ]
    smart = reduce(lambda a,b:a.unionByName(b),smart_list)

    path = out_dir + f'smart_{t}'
    smart.write.mode('overwrite').parquet(path)
    smart = spark.read.parquet(path)
    
    for i in (1,2,3):
        smart_sum = (
            smart
            .groupBy(f'count_{i}')
            .agg(F.count('title_num').alias('true_num'))
        )
        smart_sum.orderBy(f'count_{i}').show(100)


5
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      36|
|      1|      45|
|      2|      18|
|      3|       1|
+-------+--------+

+-------+--------+
|count_2|true_num|
+-------+--------+
|      0|      47|
|      1|      44|
|      2|       8|
|      3|       1|
+-------+--------+

+-------+--------+
|count_3|true_num|
+-------+--------+
|      0|      51|
|      1|      38|
|      2|      11|
+-------+--------+

10
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      43|
|      1|      40|
|      2|      12|
|      3|       5|
+-------+--------+

+-------+--------+
|count_2|true_num|
+-------+--------+
|      0|      45|
|      1|      45|
|      2|      10|
+-------+--------+

+-------+--------+
|count_3|true_num|
+-------+--------+
|      0|      47|
|      1|      42|
|      2|       9|
|      3|       2|
+-------+--------+

30
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      47|
|      1|      38|
|      2|      12

In [21]:
tl = [
    5,
    10,
    30,
    50,
    100,
    300,
    500
]

for t in tl:
    print(t)
    smart_list = [
        get_num_str_res(latest_num=x, ns=t, ntile=14)
        for x in range(1563,1463,-1)    
    ]
    smart = reduce(lambda a,b:a.unionByName(b),smart_list)

    path = out_dir + f'smart_{t}_14'
    smart.write.mode('overwrite').parquet(path)
    smart = spark.read.parquet(path)
    
    for i in [1]:
        smart_sum = (
            smart
            .groupBy(f'count_{i}')
            .agg(F.count('title_num').alias('true_num'))
        )
        smart_sum.orderBy(f'count_{i}').show(100)


5
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      49|
|      1|      39|
|      2|      12|
+-------+--------+

10
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      51|
|      1|      37|
|      2|      10|
|      3|       2|
+-------+--------+

30
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      56|
|      1|      35|
|      2|       8|
|      3|       1|
+-------+--------+

50
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      55|
|      1|      38|
|      2|       6|
|      3|       1|
+-------+--------+

100
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      60|
|      1|      33|
|      2|       7|
+-------+--------+

300
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      55|
|      1|      37|
|      2|       7|
|      3|       1|
+-------+--------+

500
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      54|
|      1|      38|
| 

In [25]:
latest_num=1563 
ns=15
ntile=10

mart = (
    number_results_ed
    .where(col('title_num') <= latest_num - 1 )
    .where(col('title_num') > latest_num - 1 - ns )
)

three_nums = (
    mart
    .groupBy('numbers')
    .agg(F.count(lit(1)).alias('count'))
    .withColumn('nums',F.row_number().over(Window.orderBy(F.desc('count'))))
    .where(col('nums')<=3)
    .agg(F.collect_list('numbers').alias('num_list'))
).take(1)[0]['num_list']
three_nums

[42, 26, 32]

In [28]:
def get_num_str_res_2(latest_num=1563, ns=30, ntile=10):
#     num_strs = ['all',f'{ns}']
    mart = (
        number_results_ed
        .where(col('title_num') <= latest_num - 1 )
        .where(col('title_num') > latest_num - 1 - ns )
    )
    
    three_nums = (
        mart
        .groupBy('numbers')
        .agg(F.count(lit(1)).alias('count'))
        .withColumn('nums',F.row_number().over(Window.orderBy(F.desc('count'))))
        .where(col('nums')<=3)
        .agg(F.collect_list('numbers').alias('num_list'))
    ).take(1)[0]['num_list']
    
    number_results_ed_ed = (
        number_results_ed
        .where(col('title_num')==latest_num)
        .withColumn('flg',F.when(col('numbers').isin(three_nums),1).otherwise(0))
    )
    
    sum_mart = (
        number_results_ed_ed
        .groupBy('title','title_num')
        .agg(
            F.sum('flg').alias('hits_count')
        )
    )
    
    path = out_dir + f'sum_mart_2_{latest_num}_{ns}_{ntile}'
#     print(path)
    sum_mart.write.mode('ignore').parquet(path)
    sum_mart = spark.read.parquet(path)
        
    return sum_mart

In [30]:
tl = [
    16,
    11,
    
]
tg = [
    (1563,1463),
    (1463,1363),
    (1363,1263),
    (1263,1163),
    (1163,1063),
    (1063,963),
]

for t in tl:
    print(t)
    for mi,ma in tg:
        print(mi,ma)
        smart_list = [
            get_num_str_res_2(latest_num=x, ns=t, ntile=14)
            for x in range(mi,ma,-1)    
        ]
        smart = reduce(lambda a,b:a.unionByName(b),smart_list)

        path = out_dir + f'smart_2_{t}_14_{mi}_{ma}'
        smart.write.mode('ignore').parquet(path)
        smart = spark.read.parquet(path)

        for i in [1]:
            smart_sum = (
                smart
                .groupBy('hits_count')
                .agg(F.count('title_num').alias('true_num'))
            )
            smart_sum.orderBy('hits_count').show(100)


16
1563 1463
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      60|
|         1|      31|
|         2|       9|
+----------+--------+

1463 1363
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      62|
|         1|      32|
|         2|       5|
|         3|       1|
+----------+--------+

1363 1263
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      61|
|         1|      34|
|         2|       5|
+----------+--------+

1263 1163
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      65|
|         1|      32|
|         2|       3|
+----------+--------+

1163 1063
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      63|
|         1|      32|
|         2|       5|
+----------+--------+

1063 963
+----------+--------+
|hits_count|true_num|
+----------+--------+
|         0|      57|
|         1|      40|
|         2|       3|
+--------

In [29]:
for t in [10,15]:
    print(t)
    path = out_dir + f'smart_{t}_14'
    smart = spark.read.parquet(path)
    
    for i in range(1,15):
        smart_sum = (
            smart
            .groupBy(f'count_{i}')
            .agg(F.count('title_num').alias('true_num'))
        )
        smart_sum.orderBy(f'count_{i}').show(100)

10
+-------+--------+
|count_1|true_num|
+-------+--------+
|      0|      51|
|      1|      37|
|      2|      10|
|      3|       2|
+-------+--------+

+-------+--------+
|count_2|true_num|
+-------+--------+
|      0|      60|
|      1|      37|
|      2|       3|
+-------+--------+

+-------+--------+
|count_3|true_num|
+-------+--------+
|      0|      64|
|      1|      34|
|      2|       2|
+-------+--------+

+-------+--------+
|count_4|true_num|
+-------+--------+
|      0|      63|
|      1|      33|
|      2|       4|
+-------+--------+

+-------+--------+
|count_5|true_num|
+-------+--------+
|      0|      62|
|      1|      34|
|      2|       4|
+-------+--------+

+-------+--------+
|count_6|true_num|
+-------+--------+
|      0|      57|
|      1|      37|
|      2|       6|
+-------+--------+

+-------+--------+
|count_7|true_num|
+-------+--------+
|      0|      71|
|      1|      22|
|      2|       7|
+-------+--------+

+-------+--------+
|count_8|true_num|
+-

ns=15, ntile=14  
で、qtile=1 の数字を買うと、2%ぐらいで当たる。（理論上、ランダムに選んだら2.5%）意味ねえ！！！！！

In [18]:
path = out_dir + f'sum_mart_2/'
sum_mart_2 = spark.read.parquet(path)
sum_mart_2.printSchema()

root
 |-- title: string (nullable = true)
 |-- title_num: integer (nullable = true)
 |-- qtile_10: integer (nullable = true)
 |-- count: long (nullable = true)
 |-- latest_num: integer (nullable = true)
 |-- ns: integer (nullable = true)
 |-- ntile: integer (nullable = true)



In [19]:
sum_mart_2.show(1)

+-------------+---------+--------+-----+----------+---+-----+
|        title|title_num|qtile_10|count|latest_num| ns|ntile|
+-------------+---------+--------+-----+----------+---+-----+
|ロト6第1482回|     1482|      12|    1|      1482| 10|   14|
+-------------+---------+--------+-----+----------+---+-----+
only showing top 1 row



In [20]:
sum_mart_2.where(col('ns')==10).groupBy('qtile_10').agg(F.sum('count')).orderBy('qtile_10').show(100)

+--------+----------+
|qtile_10|sum(count)|
+--------+----------+
|       1|        63|
|       2|        43|
|       3|        38|
|       4|        41|
|       5|        42|
|       6|        49|
|       7|        36|
|       8|        51|
|       9|        45|
|      10|        44|
|      11|        35|
|      12|        37|
|      13|        37|
|      14|        39|
+--------+----------+



In [101]:
rf = (lambda: '_'.join([ str(random.choice(range(1,44))) for _ in range(3)]) )
rf()

'34_2_20'

In [102]:
import random

random_udf = F.udf(rf, StringType()).asNondeterministic()

In [103]:
n_rand = (
    number_results_ed
    .select('title')
    .distinct()
    .withColumn('rnd_s', random_udf())
    .withColumn('rand_list',F.split('rnd_s','_'))
    .drop('rnd_s')
)

In [104]:
path = out_dir + 'n_rand'
n_rand.write.mode('overwrite').parquet(path)
n_rand = spark.read.parquet(path)

In [106]:
res = (
    number_results_ed
#     .where(col('title_num')>=1463)
    .join(n_rand,'title','left_outer')
    .where( F.array_contains(col('rand_list'),col('numbers').cast('string')) )
    .select('title','numbers','rand_list')
    .distinct()
)

path = out_dir + 'res'
res.write.mode('overwrite').parquet(path)
res = spark.read.parquet(path)


# .groupBy('title').agg(F.count(lit(1)).alias('count')).where(col('count')==6).show() #.groupBy('count').agg(F.countDistinct('title')).show()

In [107]:
res.groupBy('title').agg(F.count(lit(1)).alias('count')).where(col('count')>=3).show()

+-----------+-----+
|      title|count|
+-----------+-----+
|ロト6第37回|    3|
+-----------+-----+



In [109]:
res.where(col('title')=='ロト6第37回').show()

+-----------+-------+-----------+
|      title|numbers|  rand_list|
+-----------+-------+-----------+
|ロト6第37回|      5|[21, 41, 5]|
|ロト6第37回|     21|[21, 41, 5]|
|ロト6第37回|     41|[21, 41, 5]|
+-----------+-------+-----------+



In [110]:
res.groupBy('title').agg(F.count(lit(1)).alias('count')).groupBy('count').agg(F.countDistinct('title')).show()

+-----+------------+
|count|count(title)|
+-----+------------+
|    1|         515|
|    3|           1|
|    2|          56|
+-----+------------+



In [111]:
number_results_ed.select(F.countDistinct('title')).show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                 1563|
+---------------------+



In [100]:
41 / 1563

0.026231605886116442

In [112]:
0.02 * 1/43 * 1/43 * 1/43

2.5155017797175093e-07

In [116]:
1/43 * 1/43 * 1/43 

1.2577508898587546e-05

# 予想ロジック

ns（過去回）=15, ntile=14
で、qtile=1 の数字を買うと、3つの数字は2%ぐらいで当たる。（理論上、ランダムに選んだら1.25 * 10^-5）

In [306]:
ns = 15
ntile = 14

In [307]:
cred = credentials.Certificate('./serviceAccount.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [308]:
docs = db.collection(u'number_results').order_by(u'date', direction=firestore.Query.DESCENDING).limit(1).stream()
doc_list = [doc.to_dict() for doc in docs]

In [309]:
latest_title = doc_list[0]['title']
# latest_title

In [310]:
latest_num = int(re.split(r'[第回]',latest_title)[1])
# latest_num

In [311]:
title_list = [ f'ロト6第{i}回' for i in range(latest_num,latest_num-ns,-1)] 

In [312]:
n=9
title_list_ed = [title_list[idx:idx + n] for idx in range(0,len(title_list), n)]
# title_list_ed

In [313]:
def get_title_data(tl):
    users_ref = db.collection(u'number_results').where(u'title', u'in', tl)
    docs = users_ref.stream()

    return [doc.to_dict() for doc in docs]

In [314]:
target_data = reduce(lambda a,b:a+b,[get_title_data(tl) for tl in title_list_ed]) 

In [315]:
file_name = f'data/number_results_{latest_num}.json'
with open(file_name, 'w') as f:
    json.dump(target_data, f, indent=4)
    
with open(file_name) as f:
    number_results = pd.read_json(f)


In [316]:
number_results_ed = number_results.explode('numbers')
# number_results_ed

In [319]:
num_count = number_results_ed[['numbers','title']].groupby('numbers').count().reset_index()
num_count.sort_values('title', ascending=False).head(3)

,numbers,title
23,26,5
37,42,5
4,5,4
